In [ ]:
import h5py
import shutil
import os
import glob
import numpy as np
from scipy import stats

from astropy import units as u
from astropy import constants as const

yr = u.yr.to(u.s)
M_solar = u.M_sun.to(u.g)

from injection_helpers import get_SNe, \
                              snapshot_number_from_basename, \
                              snapshot_basename_from_number, \
                              get_last_snapshot_file_in_dir, \
                              which_SN_is_about_to_explode

In [ ]:
inputs_dir = "../ICs/cluster_cooling_150/"
outputs_dir = inputs_dir.replace("ICs", "output")

In [ ]:
SNe = get_SNe()
SN_times         = np.array([SN.time          for SN in SNe])
SN_ejecta_mass   = np.array([SN.ejecta_mass   for SN in SNe])
SN_ejecta_mass_Z = np.array([SN.ejecta_mass_Z for SN in SNe])

In [ ]:
SN_times

In [ ]:
SN_ejecta_mass

In [ ]:
SN_ejecta_mass_Z

In [ ]:
snapshot_file_old = get_last_snapshot_file_in_dir(outputs_dir)
snapshot_number_old = snapshot_number_from_basename(os.path.basename(snapshot_file_old))

snapshot_file_new = os.path.join(outputs_dir, snapshot_basename_from_number(snapshot_number_old+1))


print(snapshot_file_old)
print(snapshot_file_new)

In [ ]:
f_old = h5py.File(snapshot_file_old, mode="r")
f_new = h5py.File(snapshot_file_new, mode="w")


f_new.create_group("Header")
f_new.create_group("PartType0")

In [ ]:
i_SN = which_SN_is_about_to_explode(f_old["Header"].attrs["Time"])
print(i_SN)

In [ ]:
SN = SNe[i_SN]

In [ ]:


average_particle_mass = np.sum(f_old["PartType0"]["Masses"]) / f_old["Header"].attrs["NumPart_Total"][0]
print(average_particle_mass)

num_new_particles_needed = int(np.ceil(SN.ejecta_mass / average_particle_mass))
print(num_new_particles_needed)

mass_per_new_particle = SN.ejecta_mass / num_new_particles_needed
print(mass_per_new_particle)

In [ ]:
for key in f_old["Header"].attrs:
    f_new["Header"].attrs[key] = f_old["Header"].attrs[key]

        
# This is necessary since h5py attrs don't allow slicing/element-wise changes
new_NumPart_ThisFile = f_new["Header"].attrs["NumPart_ThisFile"]
new_NumPart_Total    = f_new["Header"].attrs["NumPart_Total"]

new_NumPart_ThisFile[0] += num_new_particles_needed
new_NumPart_Total[0]    += num_new_particles_needed

f_new["Header"].attrs["NumPart_ThisFile"] = new_NumPart_ThisFile
f_new["Header"].attrs["NumPart_Total"]    = new_NumPart_Total

old_total_particles = f_old["Header"].attrs["NumPart_Total"][0]
new_total_particles = f_new["Header"].attrs["NumPart_Total"][0]


# Make other changes
f_new["Header"].attrs["Time"] = SN.time + 1e-3 # 1 kyr after the SN


# check data
for key in f_new["Header"].attrs:
    print(key, ":", f_new["Header"].attrs[key]) 

In [ ]:
for key in f_old["Header"].attrs:
    print(key, ":", f_old["Header"].attrs[key])

In [ ]:
for key in f_old["PartType0"].keys():    
    new_shape = (new_total_particles,) + f_old["PartType0"][key].shape[1:]
    dtype = f_old["PartType0"][key].dtype
    
    f_new["PartType0"].require_dataset(key, new_shape, dtype=dtype)
    
    f_new["PartType0"][key][:old_total_particles] = f_old["PartType0"][key]
    
    print(key, ":", f_new["PartType0"][key])
    
r_SN = 2 # pc    
injection_kernel = stats.norm(loc=f_new["Header"].attrs["BoxSize"]/2, scale=r_SN)
f_new["PartType0"]["Coordinates"][-num_new_particles_needed:,0] = injection_kernel.rvs(size=num_new_particles_needed)
f_new["PartType0"]["Coordinates"][-num_new_particles_needed:,1] = injection_kernel.rvs(size=num_new_particles_needed)
f_new["PartType0"]["Coordinates"][-num_new_particles_needed:,2] = injection_kernel.rvs(size=num_new_particles_needed)

f_new["PartType0"]["Density"][-num_new_particles_needed:] = np.mean(f_old["PartType0"]["Density"]) # EDIT: will this mess with my yt plots of this snapshot?

f_new["PartType0"]["ElectronAbundance"][-num_new_particles_needed:] = 1.0 # EDIT: unused?

energy_in_correct_units = 1e51 * u.erg.to(u.pc**2 * u.M_sun / u.Myr**2)
f_new["PartType0"]["InternalEnergy"][-num_new_particles_needed:] = energy_in_correct_units/SN.ejecta_mass

f_new["PartType0"]["Masses"][-num_new_particles_needed:] = mass_per_new_particle

f_new["PartType0"]["Metallicity"][-num_new_particles_needed:] = SN.ejecta_mass / SN.ejecta_mass_Z

f_new["PartType0"]["NeutralHydrogenAbundance"][-num_new_particles_needed:] = 0.0 # EDIT: unused?

f_new["PartType0"]["ParticleChildIDsNumber"][-num_new_particles_needed:] = 0 # is this right?

f_new["PartType0"]["ParticleIDGenerationNumber"][-num_new_particles_needed:] = 0 # is this right?

new_particle_ids = np.max(f_old["PartType0"]["ParticleIDs"]) + 1 + np.arange(num_new_particles_needed)
f_new["PartType0"]["ParticleIDs"][-num_new_particles_needed:] = new_particle_ids

f_new["PartType0"]["SmoothingLength"][-num_new_particles_needed:] = np.mean(f_old["PartType0"]["SmoothingLength"])

f_new["PartType0"]["Velocities"][-num_new_particles_needed:,0] = 0
f_new["PartType0"]["Velocities"][-num_new_particles_needed:,1] = 0
f_new["PartType0"]["Velocities"][-num_new_particles_needed:,2] = 0



In [ ]:
f_old.close()
f_new.close()